In [4]:

import requests
import mysql.connector
import time
import random
import selenium as se
from collections import Counter
from bs4 import BeautifulSoup
from difflib import SequenceMatcher
import unicodedata

In [5]:
cnx = mysql.connector.connect(user='david', password='daviddung1993',
                          host='127.0.0.1',
                          database='computervision')
cursor = cnx.cursor()
headers = {"x-api-key": "M7HSjQNeTfai6l7JUiDZB8XYc85BHnHt3R0NXSEd"}
#pg = ProxyGenerator()

In [3]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def preprocess_author_name(name):
    return unicodedata.normalize('NFD', name.lower().replace("ü", "ue").replace("ß", "ss").replace("ä","ae").replace("ö", "oe")).encode('ascii', 'ignore').decode("utf-8") .replace("-", " ")

In [6]:
cursor.execute("select distinct p.PaperID, p.Title, GROUP_CONCAT(b.AuthoredByID, ':',  a.Name), p.DOI, count(*) from Papers p, authoredBy b, Authors a where p.PaperID not like '1111%' and p.DOI like '%/%' and p.PaperID = b.PaperID and a.AuthorID = b.AuthoredByID group by p.PaperID, p.Title, p.DOI")
left_papers = cursor.fetchall()

In [14]:
all_prefixes = [paper[3].split("/")[0] for paper in left_papers]
all_prefix = list(set(all_prefixes))

In [15]:
journal_prefix = {}

for idx, prefix in enumerate(all_prefix):
    if idx % 10 == 0:
        print(idx)
    try:
        resp = requests.get(f"https://api.crossref.org/prefixes/{prefix}").json()
        journal_prefix[prefix] = resp["message"]["name"]
    except ValueError:
        journal_prefix[prefix] = None

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2

In [16]:
grouped_journals = {}
grouped_journal_ids = {}
x = Counter(all_prefixes)
for k, v in x.items():
    if journal_prefix[k] not in grouped_journals:
        grouped_journals[journal_prefix[k]] = v
        grouped_journal_ids[journal_prefix[k]] = [k]
    else:
        grouped_journals[journal_prefix[k]] += v
        grouped_journal_ids[journal_prefix[k]].append(k)

In [17]:
a = sorted(grouped_journals.items(), key=lambda x: x[1], reverse=True)
a

[('Institute of Electrical and Electronics Engineers (IEEE)', 136666),
 ('Springer Science and Business Media LLC', 59425),
 ('Elsevier BV', 54838),
 ('Association for Computing Machinery (ACM)', 20566),
 (None, 14315),
 ('Wiley', 12931),
 ('MDPI AG', 6938),
 ('Informa UK Limited', 6367),
 ('SPIE-Intl Soc Optical Eng', 6324),
 ('Association for Computational Linguistics (ACL)', 3975),
 ('Association for the Advancement of Artificial Intelligence (AAAI)', 3794),
 ('Optica Publishing Group', 3279),
 ('Oxford University Press (OUP)', 3245),
 ('IOP Publishing', 2976),
 ('SAGE Publications', 2573),
 ('Society for Industrial & Applied Mathematics (SIAM)', 2324),
 ('Ovid Technologies (Wolters Kluwer Health)', 2167),
 ('Public Library of Science (PLoS)', 2095),
 ('Frontiers Media SA', 2025),
 ('Hindawi Limited', 1686),
 ('JSTOR', 1611),
 ('Institution of Engineering and Technology (IET)', 1595),
 ('MIT Press', 1465),
 ('Cambridge University Press (CUP)', 1460),
 ('International Joint Conferenc

In [21]:
for publisher in grouped_journals:
    prefixes = grouped_journal_ids[publisher]
    for prefix in prefixes:
        cursor.execute("update Papers p set p.publisher = %s where p.DOI like %s",[publisher,prefix + "/%"])
        cnx.commit()

In [18]:
grouped_journal_ids["Institute of Electrical and Electronics Engineers (IEEE)"]

['10.1109', '10.23919', '10.36227', '10.21629']

In [9]:
#grouped_journal_ids['Elsevier BV']
springer_prefix = ['10.1093']

In [10]:
for prefix in springer_prefix:
    cursor.execute("update Papers p set p.publisher ='Oxford University Press (OUP)' where p.DOI like %s",[prefix + "/%"])
    cnx.commit()

In [11]:
cursor.execute("select distinct p.PaperID, p.DOI from Papers p, authoredBy b, Authors a where p.PaperID not like '1111%' and p.publisher = 'Oxford University Press (OUP)' and p.DOI != 'NA' and p.PaperID = b.PaperID and b.AuthoredByID not in (select a.AuthorID from affiliatedTo a) and a.AuthorID = b.AuthoredByID group by p.PaperID, p.Title, p.DOI order by p.PaperID ASC")
cat_papers = cursor.fetchall()

In [12]:
len(cat_papers)

308

In [14]:
crawler = Crawler()

In [15]:
class Found(Exception): pass
for idx, paper in enumerate(cat_papers):

    if idx % 10 == 0:
        print(idx)

    cursor.execute("select distinct a.AuthorID, a.Name from Papers p,authoredBy b,Authors a where p.PaperID=%s and p.PaperID = b.PaperID and b.AuthoredByID not in (SELECT a.AuthorID from affiliatedTo a) and b.AuthoredByID = a.AuthorID",[paper[0]])
    authors = cursor.fetchall()

    if not authors or authors[0][0] == "0":
        continue


    #author_names = [author[1] for author in authors]
    print(paper)
    #affiliations = crawler.spie_request(paper[1])
    #affiliations = crawler.springer_request(paper[1], scrape=True)
    affiliations = crawler.oxford_request(paper[1])
    #affiliations = crawler.science_direct_request(paper[1], scrape=True)

    if isinstance(affiliations, str):
        print("Wrong DOI")
        cursor.execute("UPDATE Papers p SET p.publisher = %s WHERE p.PaperID=%s",[affiliations,paper[0]])
        cnx.commit()
        continue

    if not affiliations:
        time.sleep(2 + random.random())
        continue

    print(affiliations)
    print(authors)
    for author in authors:
        try:
            affiliation_country = None
            found = False
            most_similar_index = -1
            """
            author_name_tokens = author[1].lower().split()
            author_name_tokens_rev = list(reversed(author[1].lower().split()))
            similarity = [similar(author_name_tokens[-1],aff_auth[0].lower().split()[-1]) for aff_auth in affiliations]
            similarity_rev = [similar(author_name_tokens_rev[-1],aff_auth[0].lower().split()[-1]) for aff_auth in affiliations]
            most_similar_index = similarity.index(max(similarity))
            most_similar_index_rev = similarity_rev.index(max(similarity_rev))

            if similarity[most_similar_index] < 0.6:
                if similarity_rev[most_similar_index_rev] < 0.6:
                    print("continued")
                    continue
                else:
                    most_similar_index = most_similar_index_rev
                    reversed_name = " ".join(author_name_tokens_rev)
                    cursor.execute("UPDATE Authors a SET a.Name=%s WHERE a.AuthorID=%s", [reversed_name,author[0]])
            """
            try:
                author_name_tokens = preprocess_author_name(author[1]).split()
                aff_authors = [preprocess_author_name(aff_auth[0]).split() for aff_auth in affiliations]
                for author_name_token in author_name_tokens:
                    for idx, aff_author in enumerate(aff_authors):
                        if any(0.8 <= similar(author_name_token, aff) for aff in aff_author):
                            most_similar_index = idx
                            found = True
                            raise Found
            except Found:
                pass
            if not found:
                print("continued")
                continue

            print("inserted")
            all_affiliations = affiliations[most_similar_index][1].split(";;")
            if len(affiliations[most_similar_index]) == 3:
                affiliation_country = affiliations[most_similar_index][2]
            print(f"{author[1]}: {all_affiliations}")
            if not all_affiliations[0]:
                cursor.execute("INSERT INTO affiliatedTo VALUES (%s,%s)", [author[0],1])
                continue
            for affiliation in all_affiliations:
                if len(affiliation) > 255:
                    affiliation = affiliation[-254:]
                cursor.execute("SELECT * FROM Affiliations a WHERE a.Name=%s", [affiliation[:400]])
                entry = cursor.fetchone()
                if not entry:
                    cursor.execute("INSERT INTO Affiliations (Name,Country) VALUES (%s,%s)", [affiliation[:400], affiliation_country])
                    lastrowid = cursor.lastrowid
                else:
                    lastrowid = entry[0]
                cursor.execute("INSERT INTO affiliatedTo VALUES (%s,%s)", [author[0],lastrowid])
        except (AttributeError, mysql.connector.DatabaseError,  se.common.exceptions.WebDriverException):
            print("error")
            time.sleep(2 + random.random())
            continue
    time.sleep(2 + random.random())
    cnx.commit()
    print("---------")





0
('0d9a2297f9f611039819a3116a5d1537a739210e', '10.1093/ajcl/avac023')
[('Mortimer Newlin Stead Sellers', '\n')]
[('118850940', 'C. Leiteizen'), ('92690061', 'Vladimir Il Ich Lenin')]
continued
continued
---------
('14c318e9a1ace263200b9a5306e49d1821a36681', '10.1093/PLANKT/FBS094')
[('Hongsheng Bi', '\n'), ('Stuart Cook', ''), ('Hao Yu', ''), ('Mark C. Benfield', ''), ('Edward D. Houde', '')]
[('145502614', 'Stuart Cook'), ('2065007344', 'Hongsheng Bi'), ('2110751368', 'Hao Yu'), ('2587455', 'Mark Benfield'), ('93949792', 'Edward Houde')]
inserted
Stuart Cook: ['']
inserted
Hongsheng Bi: ['\n']
inserted
Hao Yu: ['']
inserted
Mark Benfield: ['']
inserted
Edward Houde: ['']
---------
('15cec157a13dac0b87963f7d6ec358b7863c1435', '10.1093/acprof:oso/9780195327939.001.0001')
[('Clancy Martin', ''), ('Robert C. Solomon', ''), ('Harry Frankfurt', ''), ('William Ian Miller', ''), ('Mark A. Wrathall', ''), ('David Sherman', ''), ('Kelly Oliver', ''), ('Paul Ekman', ''), ('Alan Strudler', ''), 

KeyboardInterrupt: 

In [14]:
options = se.webdriver.ChromeOptions()
options.add_argument("headless")
options.add_argument(
    "user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15")
browser = se.webdriver.Chrome(options=options)


In [17]:
paper = cat_papers[4]
cursor.execute("select distinct a.AuthorID, a.Name from Papers p,authoredBy b,Authors a where p.PaperID=%s and p.PaperID = b.PaperID and b.AuthoredByID not in (SELECT AuthorID from affiliatedTo) and b.AuthoredByID = a.AuthorID",[paper[0]])
authors = cursor.fetchall()
print(authors)
print(paper[0])
print(paper[1])
affiliations = scrape_request(paper[1])
print(affiliations)
return
#print(affiliations)
#print(authors)

try:
    for author in authors:
        try:
            similarity = [similar(author[1].lower().split()[-1],aff_auth[0].lower().split()[-1]) for aff_auth in affiliations]
            most_similar_index = similarity.index(max(similarity))

            if similarity[most_similar_index] < 0.6:
                print("continued")
                continue
            print("inserted")
            affiliation = affiliations[most_similar_index][1]
            if not affiliations[most_similar_index][1]:
                cursor.execute("INSERT INTO affiliatedTo VALUES (%s,%s)", [author[0],1])
                continue
            cursor.execute("SELECT * FROM Affiliations a WHERE a.Name=%s", [affiliation])
            entry = cursor.fetchone()
            if not entry:
                cursor.execute("INSERT INTO Affiliations (Name) VALUES (%s)", [affiliation[:500]])
                lastrowid = cursor.lastrowid
            else:
                lastrowid = entry[0]
            cursor.execute("INSERT INTO affiliatedTo VALUES (%s,%s)", [author[0],lastrowid])
        except AttributeError:
            continue
    cnx.commit()
    print("---------")
except:
    print("---------")
continue

[('2055029893', 'Qing Xue')]
07a1aabf83a48768945cee1be7dd7695515517db
10.1109/CVPR.2004.158
[]


SyntaxError: 'return' outside function (546858100.py, line 9)

In [11]:
def scrape_request(doi):
    all_affiliations = []
    browser.get(f"https://doi.org/{doi}")
    time.sleep(1.5 + random.random())
    browser.get(f"{browser.current_url}/authors#authors")
    source = browser.page_source
    soup = BeautifulSoup(source, 'html.parser')
    for node in soup.find_all("div", {"class": "authors-accordion-container"}):
        all_text = node.findAll(text=True)
        all_affiliations.append((all_text[1], parse_affiliation(all_text[7])))
    time.sleep(1 + random.random())
    return all_affiliations

In [21]:

browser.get(f"https://doi.org/{cat_papers[1][1]}")
time.sleep(2)
url = browser.current_url
print(url)
browser.get(f"{url}/authors#authors")
source = browser.page_source
soup = BeautifulSoup(source)
for node in soup.find_all("div", {"class":"authors-accordion-container"}):
    all_text = node.findAll(text=True)
    print(f"{all_text[1]}: {all_text[7]}")
print("-----------------------------")
time.sleep(1.5)
#browser.close()

https://ieeexplore.ieee.org/document/1512408
X. Otazu: The Centre de Visio per Computador, Universitat Authnòma de Barcelona, Barcelona, Spain
M. Gonzalez-Audicana: Department of Projects and Rural Engineering, Escuela Tecnica Superior de Ingenieros Agronomos, Public University of Navarra, Pamplona, Spain
O. Fors: Departament ďAstronomia i Meteorologia, Universitat de Barcelona, Barcelona, Spain
J. Nunez: Departament ďAstronomia i Meteorologia, Universitat de Barcelona, Barcelona, Spain
-----------------------------


hallo


In [60]:
def parse_affiliation(affiliation):
    if not affiliation:
        return "None"
    split_affiliation = affiliation.split(",")
    if len(split_affiliation) == 1:
        return affiliation
    return ",".join(split_affiliation[0:-1])

In [63]:
parse_affiliation("Germany")

'Germany'

In [9]:
requests.get("https://www.ncbi.nlm.nih.gov/pmc/utils/idconv/v1.0/.").content

b'<!DOCTYPE html\n\tPUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"\n\t "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml" lang="en-US" xml:lang="en-US">\n<head>\n<title>Any ID converter</title>\n<meta http-equiv="Content-Type" content="text/html; charset=iso-8859-1" />\n</head>\n<body bgcolor="#F8F8F8">\n<h1>Any ID converter</h1><hr /><form method="get" action="https://www.ncbi.nlm.nih.gov/pmc/utils/idconv/v1.0/" enctype="multipart/form-data"><p><textarea name="ids"  rows="15" cols="60"></textarea> <br /> <br /> Convert: <select name="idtype" >\n<option value="pmcid">pmcid</option>\n<option selected="selected" value="pmid">pmid</option>\n<option value="mid">mid</option>\n<option value="doi">doi</option>\n</select> <br /> Output:&nbsp;&nbsp; <select name="format" >\n<option value="xml">xml</option>\n<option selected="selected" value="html">html</option>\n<option value="json">json</option>\n<option value="csv">csv</option>\n</select> <

In [ ]:
response = requests.get("http://list.didsoft.com/get?email=david.le@inovex.de&pass=m75h35&pid=http3000&showcountry=no").content.decode("utf-8")
proxy_list = response.split("\n")

In [115]:
contrib_group = a["response"]["records"]["article"]["front"]["article-meta"]["contrib-group"]
all_affiliations = []
for x in contrib_group["contrib"]:
    name = f"{x['name']['given-names']} {x['name']['surname']}"
    affiliation = []
    for aff in x["xref"]:
        if aff["@ref-type"] == "aff":
            ref_id = int(aff["@rid"][-1])
            affiliation.append(contrib_group["aff"][ref_id-1]["institution-wrap"]["institution"]["#text"])

    affiliations = ";".join(affiliation)
    all_affiliations.append((name, affiliations))


In [5]:
cursor.execute("select p.PaperID, p.Title from Papers p where p.DOI is NULL")
all_papers = cursor.fetchall()
len(all_papers)

66965

In [ ]:
for idx,paper in enumerate(all_papers):
    try:
        if idx % 100 == 0:
            print(idx)
        cursor.execute("select distinct a.AuthorID, a.Name from Papers p,authoredBy b,Authors a where p.PaperID=%s and p.PaperID = b.PaperID and b.AuthoredByID = a.AuthorID",[paper[0]])
        authors = cursor.fetchall()

        if not authors:
            cursor.execute("UPDATE Papers p SET p.DOI='None' where p.PaperID=%s",[paper[0]])
            cnx.commit()
            continue
        first_name = authors[0][1].split()[-1]
        doi = research_crawler.extract_doi(paper[1],first_name)
        if doi:
            cursor.execute("UPDATE Papers p SET p.DOI=%s where p.PaperID=%s",[doi,paper[0]])
        else:
            cursor.execute("UPDATE Papers p SET p.DOI='NA' where p.PaperID=%s",[paper[0]])
        cnx.commit()
    except:
        print(paper[1])
        continue

0
Acknowledgements
Pedestrian Detection Inspired by Appearance Constancy and Shape Symmetry
Real Time Fabric Defect Detection System on an Embedded DSP Platform
100
200
